In [1]:
import kglab
kglab.__version__

'0.6.1'

In [2]:
namespaces = {
    "twokkg":  "http://w3id.org/moo/kg/2k/",
    "twokvoc": "http://w3id.org/moo/ont/2k/",
    "moo":  "http://w3id/org/moo/ont/",
    "wd": "http://www.wikidata.org/entity/",
    "wds": "http://www.wikidata.org/entity/statement/",
    "wdv": "http://www.wikidata.org/value/",
    "wdt": "http://www.wikidata.org/prop/direct/",
    "wikibase": "http://wikiba.se/ontology#",
     "p": "http://www.wikidata.org/prop/",
     "ps": "http://www.wikidata.org/prop/statement/",
     "pq": "http://www.wikidata.org/pr+op/qualifier/",
      "bd": "http://www.bigdata.com/rdf#",
    "time": "http://www.w3.org/2006/time#",
}

kg = kglab.KnowledgeGraph(
    name = "Codes Vocabulary extracted from JFMM and Ship's 3M Manual",
    base_uri = "https://w3id.org/moo/voc/mooVoc",
    namespaces = namespaces,
    )
kg.describe_ns()

,prefix,namespace
0,dct,http://purl.org/dc/terms/
1,owl,http://www.w3.org/2002/07/owl#
2,prov,http://www.w3.org/ns/prov#
3,rdf,http://www.w3.org/1999/02/22-rdf-syntax-ns#
4,rdfs,http://www.w3.org/2000/01/rdf-schema#
5,schema,http://schema.org/
6,sh,http://www.w3.org/ns/shacl#
7,skos,http://www.w3.org/2004/02/skos/core#
8,xsd,http://www.w3.org/2001/XMLSchema#
9,twokkg,http://w3id.org/moo/kg/2k/


Load in each of the .csv files with code disambiguation

In [3]:
import pandas as pd
import uuid
import rdflib
from rdflib.namespace import XSD

,Unnamed: 0,0,1
0,0,Code,Description
1,1,1,Abnormal Environment
2,2,2,Manufacturer or Installation Defects
3,3,3,Lack of Knowledge or Skill
4,4,4,Communication Problem


In [4]:
from pathlib import PurePath, Path
p = PurePath('./tables/causeCode.csv')
df = pd.read_csv(p)
pd.set_option('display.max_columns', None)
df.head(5)

In [5]:
df.columns = df.iloc[0]
display(df)
df

,0,Code,Description
0,0,Code,Description
1,1,1,Abnormal Environment
2,2,2,Manufacturer or Installation Defects
3,3,3,Lack of Knowledge or Skill
4,4,4,Communication Problem
5,5,5,Inadequate Instruction or Procedure
6,6,6,Inadequate Design
7,7,7,Normal Wear and Tear
8,8,0,Other or No Malfunction


,0,Code,Description
0,0,Code,Description
1,1,1,Abnormal Environment
2,2,2,Manufacturer or Installation Defects
3,3,3,Lack of Knowledge or Skill
4,4,4,Communication Problem
5,5,5,Inadequate Instruction or Procedure
6,6,6,Inadequate Design
7,7,7,Normal Wear and Tear
8,8,0,Other or No Malfunction


In [6]:
df = df.iloc[pd.RangeIndex(len(df)).drop(0)]


In [7]:
kg.load_rdf('./decoder-ring/codetypes.ttl')

In [8]:
#Create a build code voc function that passes in each of the csv files
def build_code_vocabulary(kg,index,row):
    #Create a top level code type
    CodeCSV = str(uuid.uuid4())
    CodeType = rdflib.URIRef("https://w3id.org/moo/voc/mooVoc{}".format(CodeCSV))
    kg.add(CodeType, kg.get_ns("rdf").type, kg.get_ns("moo").Code)
    kg.add(CodeType, kg.get_ns("rdf").type, kg.get_ns("prov").Collection)
    kg.add(CodeType, kg.get_ns("dct").title, rdflib.Literal("Naval Maintenance Code"))
    #Create Cause Code Type
    CauseCodeCSV = str(uuid.uuid4())
    CauseCodeType =  rdflib.URIRef("https://w3id.org/moo/voc/mooVoc{}".format(CauseCodeCSV))
    kg.add(CauseCodeType, kg.get_ns("rdf").type, kg.get_ns("moo").Code)
    kg.add(CodeType, kg.get_ns("dct").title, rdflib.Literal("Cause Code"))
    #Add subclass relationship to CodeType
    kg.add(CauseCodeType, kg.get_ns("rdf").subClassOf,kg.get_ns("moo").CodeType)
    #Add string literals for the dataframe
    kg.add(CauseCodeType, kg.get_ns("moo").codeText, rdflib.Literal(row['Code']))
    kg.add(CauseCodeType, kg.get_ns("moo").codeDescription, rdflib.Literal(row['Description']))
    #Adding Cause Node
    CauseCSV = str(uuid.uuid4())
    Cause =  rdflib.URIRef("https://w3id.org/moo/voc/mooVoc{}".format(CauseCSV))
    kg.add(CauseCodeType, kg.get_ns("moo").hasCause, kg.get_ns("moo").Cause)
    #Cause is stub

#passes in KG object, 

In [9]:
#Building Function:
for index, row in df.iterrows():
    build_code_vocabulary(kg,index,row)

In [10]:
# text = kg.save_rdf_text()
# print(text)
kg.save_rdf("./kg/codeTypes.ttl")